# 🤖 Train an Emotion Detection Model using FER-2013

In [ ]:
# Step 1: Upload FER-2013 dataset CSV (fer2013.csv)
from google.colab import files
uploaded = files.upload()

In [ ]:
# Step 2: Preprocess the data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

data = pd.read_csv('fer2013.csv')
pixels = data['pixels'].tolist()

faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(48, 48)
    faces.append(face.astype('float32'))

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)
faces /= 255.0

emotions = pd.get_dummies(data['emotion']).values
x_train, x_val, y_train, y_val = train_test_split(faces, emotions, test_size=0.2, random_state=42)

In [ ]:
# Step 3: Build and train the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=64)

In [ ]:
# Step 4: Save the model
model.save('emotion_model.h5')

In [ ]:
# Step 5: Download the model
from google.colab import files
files.download('emotion_model.h5')